<a href="https://colab.research.google.com/github/DMXMax/covid19/blob/master/coviddata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import io


In [0]:
dfData = pd.read_csv("/content/Coronavirus numbers by state 20200308.csv")

In [30]:
dfData.head()

,Date,State,Positive,Negative,Pending,Total,Notes
0,20200304,AZ,2.0,27,5.0,34,NaN
1,20200304,CA,53.0,462,NaN,515,NaN
2,20200304,FL,2.0,24,16.0,42,NaN
3,20200304,GA,2.0,NaN,NaN,2,NaN
4,20200304,IL,4.0,124,27.0,155,NaN


In [31]:
dfData.columns

Index(['Date', 'State', 'Positive', 'Negative', 'Pending', 'Total', 'Notes'], dtype='object')

In [32]:
dfData.State.value_counts()

NH    6
TX    6
FL    6
WA    6
AZ    6
CA    6
OR    6
MA    6
NY    6
NC    6
WI    6
GA    6
SC    6
IL    6
VA    5
CO    5
TN    5
NJ    5
NE    5
MD    5
NV    5
DC    5
OH    5
MI    5
AR    4
IN    4
NM    4
DE    4
WV    4
IA    4
PA    4
MN    4
KS    4
VT    4
AK    4
KY    4
UT    3
ME    3
AL    3
SD    3
MT    3
ND    3
HI    3
OK    3
WY    3
CT    3
RI    3
MO    3
MS    3
ID    3
LA    3
Name: State, dtype: int64

In [33]:
dfData.dtypes



Date          int64
State        object
Positive    float64
Negative     object
Pending     float64
Total         int64
Notes        object
dtype: object

In [34]:
dfData.index

RangeIndex(start=0, stop=227, step=1)

In [35]:
dfData.describe()

,Date,Positive,Pending,Total
count,2.270000e+02,225.000000,116.000000,227.000000
mean,2.020031e+07,7.924444,17.965517,65.140969
std,1.501899e+00,21.741759,36.641609,151.501738
min,2.020030e+07,0.000000,0.000000,0.000000
25%,2.020031e+07,0.000000,1.750000,5.000000
50%,2.020031e+07,1.000000,5.000000,14.000000
75%,2.020031e+07,4.000000,14.250000,45.000000
max,2.020031e+07,136.000000,236.000000,1326.000000


In [36]:
dfData.loc[:, ['State', 'Positive']].sort_values(by='Positive')

,State,Positive
112,DE,0.0
121,ND,0.0
120,MS,0.0
119,MO,0.0
118,LA,0.0
...,...,...
125,NY,105.0
180,CA,114.0
223,WA,136.0
123,AL,NaN


In [42]:
dfData[(dfData.Positive >0) & (dfData.Date==20200309)].sort_values(by='Positive').Date.count()

35

In [0]:
dfPositive= dfData[dfData.Positive>0]

In [39]:
dfPositive[dfPositive.State=='CA']

,Date,State,Positive,Negative,Pending,Total,Notes
1,20200304,CA,53.0,462,NaN,515,NaN
15,20200305,CA,53.0,462,NaN,515,NaN
39,20200306,CA,60.0,462,NaN,522,NaN
76,20200307,CA,69.0,462,NaN,531,NaN
127,20200308,CA,88.0,462,NaN,550,NaN
180,20200309,CA,114.0,690,NaN,804,Negative from CDPH report of 778 tests on 3/7


In [0]:
df2 = dfPositive.State.unique()

In [44]:
df2

array(['AZ', 'CA', 'FL', 'GA', 'IL', 'MA', 'NC', 'NH', 'NY', 'OR', 'TX',
       'WA', 'WI', 'NJ', 'NV', 'TN', 'MD', 'CO', 'PA', 'IN', 'SC', 'RI',
       'MN', 'NE', 'KY', 'HI', 'OK', 'UT', 'VA', 'CT', 'VT', 'KS', 'DC',
       'MO', 'IA'], dtype=object)

In [0]:
df1 = dfPositive[dfPositive.Date == 20200308].State
df2 = dfPositive[dfPositive.Date == 20200309].State

In [63]:
print(df1.count())
print(df2.count())

34
35


In [79]:
dfData.loc[  dfData.State=='CA',['Date', 'Positive', 'Negative', 'Pending', 'Total'] ]


,Date,Positive,Negative,Pending,Total
1,20200304,53.0,462,NaN,515
15,20200305,53.0,462,NaN,515
39,20200306,60.0,462,NaN,522
76,20200307,69.0,462,NaN,531
127,20200308,88.0,462,NaN,550
180,20200309,114.0,690,NaN,804


In [80]:
690-462

228

In [81]:
550-804

-254

In [82]:
dfData.iloc[127]

Date        20200308
State             CA
Positive          88
Negative         462
Pending          NaN
Total            550
Notes            NaN
Name: 127, dtype: object

In [83]:
dfData.iloc[180]

Date                                              20200309
State                                                   CA
Positive                                               114
Negative                                               690
Pending                                                NaN
Total                                                  804
Notes       Negative from CDPH report of 778 tests on 3/7 
Name: 180, dtype: object

In [0]:
df1 = dfData.loc[180, ['Positive', 'Negative','Total']]
df2 = dfData.loc[127, ['Positive', 'Negative','Total']]

In [129]:
dfPositive.astype({'Positive': 'int32'})


,Date,State,Positive,Negative,Pending,Total,Notes
0,20200304,AZ,2,27,5.0,34,NaN
1,20200304,CA,53,462,NaN,515,NaN
2,20200304,FL,2,24,16.0,42,NaN
3,20200304,GA,2,NaN,NaN,2,NaN
4,20200304,IL,4,124,27.0,155,NaN
...,...,...,...,...,...,...,...
220,20200309,UT,1,NaN,NaN,1,Only positives reported
221,20200309,VA,2,36,6.0,44,NaN
222,20200309,VT,1,28,NaN,29,NaN
223,20200309,WA,136,"1,110",80.0,1326,Pending from counties


In [0]:
dfClean = dfData.fillna(0)

In [172]:
dfClean.astype({'Positive': 'int32', 'Negative': 'int32', 'Total': 'int32'})

,Date,State,Positive,Negative,Pending,Total,Notes
0,20200304,AZ,2,27,5.0,34,0
1,20200304,CA,53,462,0.0,515,0
2,20200304,FL,2,24,16.0,42,0
3,20200304,GA,2,0,0.0,2,0
4,20200304,IL,4,124,27.0,155,0
...,...,...,...,...,...,...,...
222,20200309,VT,1,28,0.0,29,0
223,20200309,WA,136,1110,80.0,1326,Pending from counties
224,20200309,WI,1,31,12.0,44,0
225,20200309,WV,0,2,3.0,5,0


In [0]:
dfClean.Negative = dfClean.Negative.astype('int32')

In [177]:
dfClean.Negative.sum()

10920

In [0]:
dfClean.loc[223, ['Negative']]=1110


In [0]:
df1 = dfClean.loc[180, ['Positive','Negative','Total']]
df2 = dfClean.loc[127, ['Positive','Negative','Total']] 

In [179]:
df1


Positive    114
Negative    690
Total       804
Name: 180, dtype: object

array([114.0, '690', 804], dtype=object)

In [180]:
df1-df2


Positive     26
Negative    228
Total       254
dtype: object